In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('Files/nih/train-small.csv')

In [3]:
# Generate an array of 4 binary label values, 3 positive and 1 negative
y_true = np.array(
        [[1],
         [1],
         [1],
         [0]])
print(f"y_true: \n{y_true}")

y_true: 
[[1]
 [1]
 [1]
 [0]]


In [8]:
# model 1
y_pred_1 = 0.9*(np.ones(y_true.shape))
y_pred_1

array([[0.9],
       [0.9],
       [0.9],
       [0.9]])

In [9]:
#model 2
y_pred_2 = 0.1*(np.ones(y_true.shape))
y_pred_2

array([[0.1],
       [0.1],
       [0.1],
       [0.1]])